In [3]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-09 22:27:28


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [4]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [5]:
# --- REMI V6.1: Softer Empathy Layer ---

import random
from textblob import TextBlob

# Emotion-aware response pools (less clinical, more human)
empathy_responses = {
    "bad": [
        "I'm sorry today feels bad.",
        "That sounds like a hard day.",
        "Some days are just heavy."
    ],
    "sad": [
        "That sounds really sad.",
        "I'm here with you.",
        "I'm glad you told me."
    ],
    "tired": [
        "Feeling tired can be exhausting.",
        "Maybe your body needs some rest.",
        "We can take things slowly."
    ],
    "lonely": [
        "Feeling lonely is really hard.",
        "You're not alone right now.",
        "I'm here with you."
    ],
    "angry": [
        "That sounds frustrating.",
        "It's okay to feel upset.",
        "I hear how strong that feeling is."
    ],
    "anxious": [
        "It's okay. You're safe right now.",
        "Let's slow down together.",
        "Nothing is urgent."
    ]
}

grounding_phrases = [
    "You're safe right now.",
    "Nothing needs to be done.",
    "We can just sit for a moment."
]

chat_state = {
    "current_topic": None,
    "asked_questions": set(),
    "standing_by": False,
    "comfort_mode": False
}

print("✅ Remi V6.1 Empathy Layer Loaded")


✅ Remi V6.1 Empathy Layer Loaded


In [13]:
# --- REMI V10: The Honest Bot (Bug Fixes) ---

import random
import re
from datetime import datetime

chat_state = {
    "standing_by": False,
    "waiting_for_name": False,
    "user_profile": {"name": None}
}

# --- HELPER: Fixes the "nothing" vs "hi" bug ---
def has_word(word_list, text):
    """
    Checks for WHOLE words only. 
    'hi' will match 'hi there' but NOT 'nothing' or 'this'.
    """
    text = text.lower()
    for word in word_list:
        # \b means "word boundary" in Regex
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            return True
    return False

def get_time_greeting():
    h = datetime.now().hour
    if h < 12: return "Good morning"
    elif 12 <= h < 18: return "Good afternoon"
    return "Good evening"

def process_response(user_input):
    text = user_input.lower().strip()
    
    # --- LAYER 1: WAITING FOR NAME ---
    if chat_state["waiting_for_name"]:
        # If user refuses
        if has_word(["no", "nope", "don't", "nothing", "stop"], text):
            chat_state["waiting_for_name"] = False
            return "Okay. I won't ask for your name. We can just chat."
        
        # Assume input is the name
        name = text.split()[0].capitalize() # Take first word
        chat_state["user_profile"]["name"] = name
        chat_state["waiting_for_name"] = False
        return f"Nice to meet you, {name}."

    # --- LAYER 2: HANDLING "UPSET" / "DON'T ASK" ---
    # This must come BEFORE greetings
    if has_word(["stop", "upset", "angry", "don't", "quiet"], text):
        chat_state["standing_by"] = True
        return "I hear you. I will stop asking questions now. I am here if you change your mind."

    # --- LAYER 3: REALITY CHECK (Weather/Location) ---
    if has_word(["rain", "raining", "sun", "sunny", "weather", "outside"], text):
        return "I live inside this computer, so I can't see the sky. But I can check the time! If you look out the window, what do you see?"

    if has_word(["time", "clock"], text):
        now = datetime.now().strftime("%I:%M %p")
        return f"My internal clock says it is {now}."

    # --- LAYER 4: GREETINGS (Now Fixed!) ---
    # Uses has_word so "nothing" doesn't trigger "hi"
    if has_word(["hello", "hi", "hey", "greetings"], text):
        greeting = get_time_greeting()
        name = chat_state["user_profile"]["name"]
        
        if name:
            return f"{greeting}, {name}."
        else:
            chat_state["waiting_for_name"] = True
            return f"{greeting}. I don't know your name yet. What is it?"

    # --- LAYER 5: ACTIVITIES ---
    if has_word(["walk", "walking"], text):
        return "A walk sounds lovely. Do you prefer walking in the city or the park?"
    
    if has_word(["hungry", "food", "eat"], text):
        return "You should get something to eat. I can wait here."

    # --- LAYER 6: FALLBACKS ---
    # If user says "nothing" or "no", just acknowledge it
    if has_word(["nothing", "no", "nope"], text):
        return "Okay. We can just sit quietly."

    return "I am listening."

# -----------------------------
# INTERACTIVE LOOP
# -----------------------------
print("--- Remi V10 (Bug Fixed) ---")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Goodbye.")
        break
        
    response = process_response(user_input)
    print(f"Remi: {response}")

--- Remi V10 (Bug Fixed) ---


You:  knock knock


Remi: I am listening.


You:  hello


Remi: Good evening. I don't know your name yet. What is it?


You:  I am peter


Remi: Nice to meet you, I.


You:  what is your name?


Remi: I am listening.


KeyboardInterrupt: Interrupted by user

The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.

We now want to introduce long term memory to our bot

For the next phases, we will do the following: 
Phase 1 (Done): Rule-Based Logic and it's about understanding how input/output works.

Phase 2 (NOW): Data Creation. We need to create a "textbook" to teach the bot.

Phase 3 (Next): Vectorization. Converting words into numbers (Math).

Phase 4 (The Goal): Training a Neural Network. Using TensorFlow/Keras to build a model that learns from your data.

Step 1: Creating the "Textbook" (Training Data)
In Machine Learning, we don't write if statements. We create Intents.

Intent: What the user means (e.g., greeting, sadness, reminiscence_childhood).

Patterns: Examples of how humans say it.

Responses: What the bot should say back.

We need to create a file called intents.json. This will be the source code for your AI's brain.

In [14]:
import json

# This is the "Textbook" we will feed to the Neural Network
data = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up", "Hey there"],
      "responses": ["Hello!", "Good to see you again", "Hi there, how can I help?"],
      "context": [""]
    },
    {
      "tag": "goodbye",
      "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "quit"],
      "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],
      "context": [""]
    },
    {
      "tag": "identity",
      "patterns": ["who are you", "what are you", "what is your name", "are you human"],
      "responses": ["I am Remi, your memory companion.", "I am a bot designed to help you share your stories."],
      "context": [""]
    },
    {
      "tag": "mood_sad",
      "patterns": ["I feel sad", "I am lonely", "I feel bad", "I am upset", "Everything is wrong", "I am depressed"],
      "responses": ["I am sorry to hear that. I am here to listen.", "It's okay to feel down sometimes. Do you want to talk about it?"],
      "context": [""]
    },
    {
      "tag": "reminisce_childhood",
      "patterns": ["I was young", "When I was a kid", "My childhood", "I grew up in", "school days"],
      "responses": ["Childhood memories are precious. What was your favorite game to play back then?", "Tell me about the house you grew up in."],
      "context": [""]
    },
    {
      "tag": "confusion",
      "patterns": ["Where am I?", "I don't know who I am", "I am lost", "What is happening"],
      "responses": ["You are safe. You are chatting with Remi.", "Take a deep breath. You are safe right here."],
      "context": [""]
    }
  ]
}

# Save this data to a file on your D: drive
with open('intents.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("✅ Training Data Created: 'intents.json'")
print("We have stopped writing rules. We are now ready for Machine Learning.")

✅ Training Data Created: 'intents.json'
We have stopped writing rules. We are now ready for Machine Learning.


We will develop a Deep Learning system and we will do it in the following steps:

Step 1: The Translation Layer (Text to Numbers)Before we train the brain, we must translate English into a format the brain understands. Neural Networks do not understand text; they only understand Matrices (Numbers).We need to perform Data Preprocessing. This involves three specific steps:

i)Tokenization: Breaking sentences into individual words.

ii)Stemming: cutting words down to their root form (e.g., "walking", "walked", "walks" $\rightarrow$ "walk"). This stops the bot from thinking "walked" is a completely different concept from "walk".

iii)Bag of Words: Converting those stems into a list of 0s and 1s.

Step 2: Preparing the data: loading the intents.json file we created.

In [17]:
!pip install tensorflow


In [20]:
import nltk

# Download the specific missing resource
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [21]:
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

# Initialize the Lemmatizer (The tool that cuts words to their root)
lemmatizer = WordNetLemmatizer()

# Download necessary NLTK data (just in case)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

# Load the "Textbook" we created
with open('intents.json') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

# Loop through every sentence in our intents file
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # 1. Tokenize: Split sentence into words
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        
        # 2. Add to documents: Associate patterns with their tag
        documents.append((word_list, intent['tag']))
        
        # 3. Add to classes: Keep track of unique tags (e.g., "greeting", "sadness")
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# 4. Lemmatize: Lowercase and strip words to root form
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words)) # Remove duplicates

classes = sorted(set(classes))

print(f"✅ Data Processed Successfully!")
print(f"• {len(documents)} Documents (Sentences)")
print(f"• {len(classes)} Classes (Intents)")
print(f"• {len(words)} Unique Lemmatized Words")

# Save these for later use (Pickling)
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

✅ Data Processed Successfully!
• 33 Documents (Sentences)
• 6 Classes (Intents)
• 52 Unique Lemmatized Words


The next step we will do 2 things:

1. Create Training Data: Convert our text into "Bags of Words" (arrays of 0s and 1s) so the math works.

2. Build & Train the Neural Network: We will use Keras (part of TensorFlow) to build a 3-layer neural network and train it on your data.

In [ ]:
#Training the model
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

# --- PART 1: CREATE TRAINING DATA ---

training = []
# Create an empty array for our output (e.g., [0, 0, 0, 0, 0, 0])
output_empty = [0] * len(classes)

for doc in documents:
    # Initialize our bag of words
    bag = []
    pattern_words = doc[0]
    # Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    # Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# Create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("✅ Training Data Created")

# --- PART 2: BUILD THE NEURAL NETWORK ---

model = Sequential()
# Layer 1: Input Layer (128 neurons, looks at our Bag of Words)
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5)) # Prevents overfitting

# Layer 2: Hidden Layer (64 neurons)
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Layer 3: Output Layer (Number of neurons = number of intents)
# Softmax gives us a % probability for each tag
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print("✅ Model Built. Starting Training...")

# --- PART 3: TRAIN THE MODEL ---

# We fit the model to the data. 
# Epochs = How many times it sees the data (200 is standard)
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Save the trained model
model.save('remi_model.h5')
print("✅ Remi is trained and saved as 'remi_model.h5'!")

✅ Training Data Created


C:\Users\ADMIN\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ADMIN\anaconda\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


✅ Model Built. Starting Training...
Epoch 1/200
